# manipulate database

> Functions to add, delete and update data in the database.

In [ ]:
#| default_exp manipulate_db

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from pathlib import Path
from httpx import get as httpx_get
from fasthtml.common import *

from keybindings_fps.create_db import *

In [ ]:
#|export
def add_binding(db, game_name: str, action_name: str, key_name: str, modifier_name: str = 'tap'):
    """Add a key binding for a specific game and action"""
    game = next(db.t.games.rows_where("name = ?", [game_name]), None)
    if not game:
        raise ValueError(f"Game '{game_name}' not found")
        
    action = next(db.t.actions.rows_where("name = ?", [action_name]), None)
    if not action:
        raise ValueError(f"Action '{action_name}' not found")
        
    key = next(db.t.game_keys.rows_where("name = ?", [key_name]), None)
    if not key:
        raise ValueError(f"Key '{key_name}' not found")
        
    modifier = next(db.t.modifiers.rows_where("name = ?", [modifier_name]), None)
    if not modifier:
        raise ValueError(f"Modifier '{modifier_name}' not found")
        
    return db.t.bindings.insert(dict(
        game_id=game['id'],
        action_id=action['id'],
        key_id=key['id'],
        modifier_id=modifier['id']
    ))


In [ ]:
#| export
def upsert_game(name: str, game_type: str = None, image_url: str = None):
    """Update existing game or insert new one if it doesn't exist"""
    # Try to find existing game with this name
    existing = db.t.games.rows_where("name = ?", [name])
    game = next(existing, None)  # Get first match or None
    
    if image_url:
        response = httpx_get(image_url)
        if response.is_success:
            image = response.content
    else:
        image = None
    
    game_data = {"name":name, "game_type":game_type, "image":image}

    if game:
        # Update existing game
        game_data['id'] = game['id']
        return db.t.games.update(game_data)
    else:
        # Add new game
        return db.t.games.insert(game_data)


# Example usage for existing database on disk

First connect to the database.

In [ ]:
#| export
db = init_db()

Check for tables and if tables have data.

In [ ]:
#| export
print(db.t)
print(db.t.actions())

actions, bindings, categories, game_keys, games, modifiers, sqlite_stat1, sqlite_stat4
[{'id': 1, 'name': 'Forward', 'description': None, 'category_id': 1}, {'id': 2, 'name': 'Backward', 'description': None, 'category_id': 1}, {'id': 3, 'name': 'Left', 'description': None, 'category_id': 1}, {'id': 4, 'name': 'Right', 'description': None, 'category_id': 1}, {'id': 5, 'name': 'Jump/climb', 'description': None, 'category_id': 1}, {'id': 6, 'name': 'Crouch', 'description': None, 'category_id': 1}, {'id': 7, 'name': 'Prone', 'description': None, 'category_id': 1}, {'id': 8, 'name': 'Sprint', 'description': None, 'category_id': 1}, {'id': 9, 'name': 'Walk', 'description': None, 'category_id': 1}, {'id': 10, 'name': 'Lean left', 'description': None, 'category_id': 1}, {'id': 11, 'name': 'Lean right', 'description': None, 'category_id': 1}, {'id': 12, 'name': 'Fire primary', 'description': None, 'category_id': 2}, {'id': 13, 'name': 'Aim', 'description': None, 'category_id': 2}, {'id': 14, 'n

# Add game to database

In [ ]:
#| hide
db.t.games()

[{'id': 1, 'name': 'default', 'game_type': 'template', 'image': None}]

In [ ]:
#| hide
game = upsert_game(name='Insurgency Sandstorm', game_type='tactical', image_url='https://assets-prd.ignimgs.com/2021/09/24/insurgency-sandstorm-button-fin-1632454496057.jpg')
db.t.games()

[{'id': 1, 'name': 'default', 'game_type': 'template', 'image': None},
 {'id': 2,
  'name': 'Insurgency Sandstorm',
  'game_type': 'tactical',
  'image': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xe2\x02\x1cICC_PROFILE\x00\x01\x01\x00\x00\x02\x0clcms\x02\x10\x00\x00mntrRGB XYZ \x07\xdc\x00\x01\x00\x19\x00\x03\x00)\x009acspAPPL\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf6\xd6\x00\x01\x00\x00\x00\x00\xd3-lcms\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\ndesc\x00\x00\x00\xfc\x00\x00\x00^cprt\x00\x00\x01\\\x00\x00\x00\x0bwtpt\x00\x00\x01h\x00\x00\x00\x14bkpt\x00\x00\x01|\x00\x00\x00\x14rXYZ\x00\x00\x01\x90\x00\x00\x00\x14gXYZ\x00\x00\x01\xa4\x00\x00\x00\x14bXYZ\x00\x00\x01\xb8\x00\x00\x00\x14rTRC\x00\x00\x01\xcc\x00\x00\x00@gTRC\x00\x00\x01\xcc\x00\x00\x

## Add default bindings to game

In [1]:
#| export
def copy_default_bindings(new_game_name: str):
    """Copy all bindings from default game to a new game"""
    # Get the new game
    new_game = next(db.t.games.rows_where("name = ?", [new_game_name]), None)
    if not new_game:
        raise ValueError(f"Game '{new_game_name}' not found")
    
    # Get default game
    default_game = next(db.t.games.rows_where("name = ?", ["default"]), None)
    if not default_game:
        raise ValueError("Default game template not found")
    
    # Get all default bindings
    default_bindings = db.t.bindings.rows_where("game_id = ?", [default_game['id']])
    
    # Copy each binding to new game
    for binding in default_bindings:
        db.t.bindings.insert({
            'game_id': new_game['id'],
            'action_id': binding['action_id'],
            'key_id': binding['key_id'],
            'modifier_id': binding['modifier_id']
        })

### Compare bindings of game with the defaults

In [1]:
#| export
def compare_with_default(game_name: str):
    """Compare a game's bindings with default bindings and return differences"""
    # Get both games
    game = next(db.t.games.rows_where("name = ?", [game_name]), None)
    default = next(db.t.games.rows_where("name = ?", ["default"]), None)
    
    if not game:
        raise ValueError(f"Game '{game_name}' not found")
    if not default:
        raise ValueError("Default game template not found")
    
    # Get bindings for both games
    game_bindings = db.t.bindings.rows_where("game_id = ?", [game['id']])
    default_bindings = db.t.bindings.rows_where("game_id = ?", [default['id']])
    
    # Create dictionaries with action_id as key for easy comparison
    game_dict = {b['action_id']: (b['key_id'], b['modifier_id']) for b in game_bindings}
    default_dict = {b['action_id']: (b['key_id'], b['modifier_id']) for b in default_bindings}
    
    # Find differences
    differences = {}
    for action_id, (key_id, mod_id) in game_dict.items():
        if action_id in default_dict and (key_id, mod_id) != default_dict[action_id]:
            differences[action_id] = {
                'game': (key_id, mod_id),
                'default': default_dict[action_id]
            }
    
    return differences

In [ ]:
#| hide
db.close()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()